Now we parse the content of the JSON file with the QASM circuits

In [5]:
using JSON
using ITensors



In [6]:

# Hadamard Gate
function h_gate(sites, i::Int)
    N = length(sites)
    h_op = op("H", sites[i])
    for k in 1:N
        if k != i
            h_op = h_op * op("Id", sites[k])
        end
    end
    return MPO(h_op, sites)
end

# Rz Gate with angle θ
function rz_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    rz_op = op("Rz", sites[i]; θ=θ)
    for k in 1:N
        if k != i
            rz_op = rz_op * op("Id", sites[k])
        end
    end
    return MPO(rz_op, sites)
end

# Rx Gate with angle θ
function rx_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    rx_op = op("Rx", sites[i]; θ=θ)
    for k in 1:N
        if k != i
            rx_op = rx_op * op("Id", sites[k])
        end
    end
    return MPO(rx_op, sites)
end

# Ry Gate with angle θ
function ry_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    ry_op = op("Ry", sites[i]; θ=θ)
    for k in 1:N
        if k != i
            ry_op = ry_op * op("Id", sites[k])
        end
    end
    return MPO(ry_op, sites)
end

# Rxx Gate with angle ϕ
function rxx_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    rxx_op = op("Rxx", sites[i], sites[j]; ϕ=ϕ)
    for k in 1:N
        if k != i && k != j
            rxx_op = rxx_op * op("Id", sites[k])
        end
    end
    return MPO(rxx_op, sites)
end

# Rzz Gate with angle ϕ
function rzz_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    rzz_op = op("Rzz", sites[i], sites[j]; ϕ=ϕ)
    for k in 1:N
        if k != i && k != j
            rzz_op = rzz_op * op("Id", sites[k])
        end
    end
    return MPO(rzz_op, sites)
end

# Ryy Gate with angle ϕ
function ryy_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    ryy_op = op("Ryy", sites[i], sites[j]; ϕ=ϕ)
    for k in 1:N
        if k != i && k != j
            ryy_op = ryy_op * op("Id", sites[k])
        end
    end
    return MPO(ryy_op, sites)
end

# CX (Controlled-X) Gate
function cx_gate(sites, control::Int, target::Int)
    N = length(sites)
    cx_op = op("CX", sites[control], sites[target])
    for k in 1:N
        if k != control && k != target
            cx_op = cx_op * op("Id", sites[k])
        end
    end
    return MPO(cx_op, sites)
end

# Projector onto |0⟩ (|0⟩⟨0|)
function proj_0_gate(sites, i::Int)
    N = length(sites)
    proj0_op = op("Proj0", sites[i])
    for k in 1:N
        if k != i
            proj0_op = proj0_op * op("Id", sites[k])
        end
    end
    return MPO(proj0_op, sites)
end

# Projector onto |1⟩ (|1⟩⟨1|)
function proj_1_gate(sites, i::Int)
    N = length(sites)
    proj1_op = op("Proj1", sites[i])
    for k in 1:N
        if k != i
            proj1_op = proj1_op * op("Id", sites[k])
        end
    end
    return MPO(proj1_op, sites)
end


proj_1_gate (generic function with 1 method)

In [7]:
function create_sites(n::Int)
    return [Index(2, "Qubit,i$i") for i in 1:n]
end
# check if the MPO creation was sucessful
function inspect_mpo(mpo::MPO, sites::Vector)
    N = length(sites) 

    println("\nManual Inspection of MPO:")
    for site_idx in 1:N
        println("Site $site_idx Operator:")
        println(mpo[site_idx])
    end
end

inspect_mpo (generic function with 1 method)

In [8]:
function parse_subcircuits(json_filename::String)
    json_data = JSON.parsefile(json_filename)
    
    subexp_dict = Dict{String, Vector{Dict{String, Any}}}()
    qubit_num_dict = Dict{String, Int}()
    
    for entry in json_data
        category = entry["Subexperiment"]
        qubit_num = entry["Qubit number"]
        qubit_range = entry["Qubit range"]
        index = entry["Subcircuit"]
        operations_raw = entry["Operations"]
        
        offset = qubit_range[1] - 1

        # Ensure consistency of qubit numbers within the same category
        if haskey(qubit_num_dict, category)
            if qubit_num_dict[category] != qubit_num
                error("Inconsistent qubit numbers in category $category")
            end
        else
            qubit_num_dict[category] = qubit_num
        end
        
        processed_operations = Vector{Dict{String, Any}}()
        for op in operations_raw
            name = op["Name"]
            qubits = op["Qubits"]
            angle_array = op["Angle"]
            
            adjusted_qubits = [q - offset for q in qubits]
            
            angle = if length(angle_array) == 1
                        angle_array[1]
                    else
                        NaN  
                    end
            
            processed_op = Dict{String, Any}(
                "Name" => name,
                "Angle" => angle,
                "Qubits" => adjusted_qubits
            )
            
            push!(processed_operations, processed_op)
        end
        
        subcircuit = Dict{String, Any}(
            "Subcircuit" => index + 1,
            "Operations" => processed_operations
        )
        
        # Add the subcircuit to the subexp_dict
        if haskey(subexp_dict, category)
            push!(subexp_dict[category], subcircuit)
        else
            subexp_dict[category] = [subcircuit]
        end
    end
    
    sorted_categories = sort(collect(keys(subexp_dict)))
    
    organized_data = Dict{String, Dict{String, Any}}()
    
    for category in sorted_categories
        sorted_subcircuits = sort(subexp_dict[category], by = sc -> sc["Subcircuit"])
        organized_data[category] = Dict{String, Any}(
            "Qubit number" => qubit_num_dict[category],
            "Subcircuits" => sorted_subcircuits
        )
    end
    
    return organized_data
end

#Fixed qubit range to always be [1, n_max], and fixed subcircuit index to start at 1

json_filename = "subcircuits.json"  
circuits_data = parse_subcircuits(json_filename)


Dict{String, Dict{String, Any}} with 2 entries:
  "B" => Dict("Qubit number"=>5, "Subcircuits"=>Dict{String, Any}[Dict("Subcirc…
  "A" => Dict("Qubit number"=>5, "Subcircuits"=>Dict{String, Any}[Dict("Subcirc…

In [ ]:
function build_mpo_sequence(sites, operations)
    """
    Builds a list of Matrix Product Operators (MPOs) based on the provided operations.
    """
    mpo_list = []

    for op_dict in operations
        gate_type = op_dict["Name"]
        qubits = op_dict["Qubits"]
        angle = get(op_dict, "Angle", NaN) 

        if gate_type in basis_gates
            if gate_type == "h"
                i = qubits[1]
                push!(mpo_list, h_gate(sites, i))

            elseif gate_type == "rx"
                θ = angle
                i = qubits[1]
                push!(mpo_list, rx_gate(sites, i, θ))

            elseif gate_type == "ry"
                θ = angle
                i = qubits[1]
                push!(mpo_list, ry_gate(sites, i, θ))

            elseif gate_type == "rz"
                θ = angle
                i = qubits[1]
                push!(mpo_list, rz_gate(sites, i, θ))

            elseif gate_type == "rxx"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, rxx_gate(sites, i, j, φ))

            elseif gate_type == "rzz"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, rzz_gate(sites, i, j, φ))

            elseif gate_type == "ryy"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, ryy_gate(sites, i, j, φ))

            elseif gate_type == "cx"
                control, target = qubits[1], qubits[2]
                push!(mpo_list, cx_gate(sites, control, target))

            end

        elseif gate_type == "measure"
            i = qubits[1]
            push!(mpo_list, proj_0_gate(sites, i))
            push!(mpo_list, -proj_1_gate(sites, i))

        else
            error("Unsupported gate type: $gate_type. Supported gates are: $(join(basis_gates, ", ")), Measure")
        end
    end

    return mpo_list
end



# New combined function to build MPO sequences and apply them to generate rhos
function mpo_sequence_apply(circ_data)
    """
    Builds dictionaries of MPO sequences and applies them to the corresponding density matrices
    for all labels and subcircuits in circ_data.
    I keep the sequence in case it's useful
    """
    all_mpo_sequences = Dict{String, Vector{Vector{MPO}}}()    
    all_rhos = Dict{String, Vector{MPO}}()                     

    for label in keys(circ_data)

        n_qubits = circ_data[label]["Qubit number"]
        sites_label = create_sites(n_qubits)

        initial_psi = productMPS(sites_label, "0")
        initial_rho = outer(initial_psi, initial_psi')

        label_mpo_sequences = Vector{Vector{MPO}}()            
        label_rhos = Vector{MPO}()                            

        for subcircuit in circ_data[label]["Subcircuits"]
            operations = subcircuit["Operations"]
            index = subcircuit["Subcircuit"]

            mpo_sequence = build_mpo_sequence(sites_label, operations)
            push!(label_mpo_sequences, mpo_sequence)

            rho = initial_rho
            
            println("Applying MPOs for subcircuit $label$index...")
            for mpo in mpo_sequence
                rho = apply(apply(mpo, rho), dag(mpo))
            end

            push!(label_rhos, rho)
        end

        all_mpo_sequences[label] = label_mpo_sequences
        all_rhos[label] = label_rhos
    end

    return (all_mpo_sequences, all_rhos)
end

basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]
all_mpo_sequences, all_rhos = mpo_sequence_apply(circuits_data);


Applying MPOs for subcircuit B1...
Applying MPOs for subcircuit B2...
Applying MPOs for subcircuit B3...
Applying MPOs for subcircuit B4...
Applying MPOs for subcircuit B5...
Applying MPOs for subcircuit B6...
Applying MPOs for subcircuit A1...
Applying MPOs for subcircuit A2...
Applying MPOs for subcircuit A3...
Applying MPOs for subcircuit A4...
Applying MPOs for subcircuit A5...
Applying MPOs for subcircuit A6...


# TODO correct measuring scheme


In [6]:
# Ensure 'sites' is used consistently
sites = create_sites(n_qubits)

# Build the MPO sequence with the same 'sites'
sequence = build_mpo_sequence(sites, gates);
# Create the MPS using the same 'sites'
psi = productMPS(sites, "0")
rho = outer(psi,psi')
maxdim1 = 1000;  # Adjust depending on memory constraints
cutoff1 = 1e-12;

LoadError: UndefVarError: `n_qubits` not defined

In [ ]:

# Ensure 'sites' is used consistently
sites = create_sites(n_qubits)

# Build the MPO sequence with the same 'sites'
sequence = build_mpo_sequence(sites, gates);
# Create the MPS using the same 'sites'
psi = productMPS(sites, "0")
rho = outer(psi,psi')
for mpo in sequence
    println("Applying MPO...")
    
    # Ensure the MPO and MPS share the same site indices
    rho = apply(apply(mpo, rho), dag(mpo))
end


LoadError: DimensionMismatch: lengths of MPOs A (5) and B (1) do not match

In [ ]:
inspect_mpo(mpos_test[], sites)


Manual Inspection of MPO:
Site 1 Operator:
ITensor ord=2
Dim 1: (dim=2|id=622|"Qubit,i1")'
Dim 2: (dim=2|id=622|"Qubit,i1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.7071067811865475   0.7071067811865475
 0.7071067811865475  -0.7071067811865475


In [ ]:
n_qubits = 3
sites = create_sites(n_qubits)
gates_test = [(:h, i) for i in 1:n_qubits]
push!(gates_test, (:measure_qpd, 1))
mpos_test = build_mpo_sequence(sites, gates_test);

psi = productMPS(sites, "0")
rho = outer(psi,psi')


Z_sum_op = ITensor()

os = OpSum()
for i in 1:n_qubits
    os += "X", i
end
Zmpo = MPO(os, sites)

rho1 = apply(apply(mpos_test[1], rho), mpos_test[1])
for mpo in mpos_test[2:end]
    # Ensure the MPO and MPS share the same site indices
    rho1 = rho1 + apply(apply(mpo, rho), dag(mpo))
end

In [ ]:
inner(rho, Zmpo)

0.0